In [ ]:
called_bc_file = "/mnt/data/jianing/darlin_paper/snakemake_DARLIN/jobs/20220815_LL837_10X_3A/DARLIN/python_DARLIN/LL837-LF-TA_S6/called_barcodes_by_SW_method.csv"
locus    = "Tigre_2022_v2" # 'Tigre_2022_v2' 'Rosa_v2' 'cCARLIN'
R1_file  = "/mnt/data/jianing/darlin_paper/snakemake_DARLIN/jobs/20220815_LL837_10X_3A/slim_fastq/LL837-LF-TA_S6_L001_R1_001.fastq.gz"
R2_file  = "/mnt/data/jianing/darlin_paper/snakemake_DARLIN/jobs/20220815_LL837_10X_3A/slim_fastq/LL837-LF-TA_S6_L001_R2_001.fastq.gz"

In [ ]:
## Matlab -> python. uisng clone_id identified by the python-version of the snakemake_DARLIN pipeline, 
## convert to raw fastq file, then run matlab version to get the alignment results.
import os
import sys
import gzip
import random
import pandas as pd
from mosaiclineage import DARLIN as dar

def gen_umi(length: int) -> str:
    bases = ['A', 'T', 'C', 'G']
    umi_sequence = ''.join(random.choice(bases) for _ in range(length))
    return umi_sequence

def gen_base_qual_score(length: int) -> str:
    qual_scores = [chr(random.randint(30, 40) + 33) for _ in range(length)]
    return ''.join(qual_scores)

def gen_10xv3_R1(seq_id: str, cell_barcode: str) -> str:
    if len(cell_barcode) != 16:
        raise ValueError("Cell barcode must be 12 bases long")
    umi_sequence = gen_umi(12)
    sequence = cell_barcode + umi_sequence
    qual_score = gen_base_qual_score(28)
    fastq_read = f"@{seq_id}\n{sequence}\n+\n{qual_score}"
    return fastq_read

def gen_10xv3_R1_fastq(cell_barcodes: list, filename: str) -> None:
    """
    Generates a compressed FASTQ file containing reads for each cell barcode provided.

    Parameters:
        cell_barcodes (list): A list of cell barcodes, each 12 bases long.
        filename (str): The output filename for the compressed FASTQ file (e.g., "R1_reads.fastq.gz").

    Raises:
        ValueError: If any cell barcode in the list is not 12 bases long.

    The function creates a gzip-compressed FASTQ file where each read is generated using the provided cell barcodes.
    The sequence ID for each read is derived from the filename prefix and the index of the barcode in the list.
    """
    prefix = filename.split('/')[-1].replace("_L001_R1_001.fastq.gz", "")
    with gzip.open(filename, 'wt') as f:
        for i, barcode in enumerate(cell_barcodes):
            if len(barcode) != 16:
                raise ValueError(f"Cell barcode at index {i} must be 12 bases long")
            seq_id = f"{prefix}_{i+1}"
            fastq_read = gen_10xv3_R1(seq_id, barcode)
            f.write(fastq_read + '\n')

def gen_10xv3_R2_fastq(sequences: list, filename: str) -> None:
    """
    Generates a compressed FASTQ file containing R2 reads for each sequence provided.

    Parameters:
        sequences (list): A list of sequences to be included in the R2 reads.
        filename (str): The output filename for the compressed FASTQ file (e.g., "R2_reads.fastq.gz").
    """
    prefix = filename.split('/')[-1].replace("_L001_R2_001.fastq.gz", "")
    with gzip.open(filename, 'wt') as f:
        for i, sequence in enumerate(sequences):
            seq_id = f"{prefix}_{i+1}"
            qual_score = gen_base_qual_score(len(sequence))
            fastq_read = f"@{seq_id}\n{sequence}\n+\n{qual_score}"
            f.write(fastq_read + '\n')

def gen_10xv3_fastq(locus: str, cell_bc: list, clone_bc: list, R1_file: str, R2_file: str) -> None:
    if locus.startswith("Tigre"):
        seq_5prime = 'GCTCGGTACCTCGCGAA'
        seq_3prime = 'GTCTTGTCGGTGCCT'
    elif locus.startswith("cCARLIN"):
        seq_5prime = 'GAGCTGTACAAGTAAGCGGC'
        seq_3prime = 'AGAATTCTAACTAGAGCTCGCTGATCAGCCT'
    elif locus.startswith("Rosa"):
        seq_5prime = 'ATGTACAAGTAAAGCGGCC'
        seq_3prime = 'GTCTGCTGTGTGCCT'
    else:
        raise ValueError(f"Invalid locus: {locus}. locus must start with ['cCARLIN','Tigre','Rosa']")
    clone_bc = [seq_5prime + i + seq_3prime for i in clone_bc]
    gen_10xv3_R1_fastq(cell_bc,  R1_file)
    gen_10xv3_R2_fastq(clone_bc, R2_file)


#### main ####
data_df = pd.read_csv(called_bc_file)
n_lines = len(data_df['clone_id'])
n_uniq_clone_bcs = len(data_df['clone_id'].unique())
msg = f'{n_lines} lines. {n_uniq_clone_bcs} unique clone barcodes'
print(msg)

data_df_uniq = data_df[['cell_bc', 'clone_id']].drop_duplicates('clone_id')
cell_barcodes = data_df_uniq['cell_bc'].tolist()
clone_barcodes = data_df_uniq['clone_id'].tolist()

gen_10xv3_fastq(
    locus    = locus, 
    cell_bc  = cell_barcodes, 
    clone_bc = clone_barcodes,
    R1_file = R1_file,
    R2_file = R2_file
)

# python ../../bin/extract_CARLIN_sequence.py ./ Tigre_2022_v2 LL837-LF-TA_S6